___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Victor Arruda Niubó

#### Giovanni Pertence Cardoso dos Santos

#### Gustavo Schlieper Tessitore

#### Leonardo Cesari Costa
___

# Previsão da quantidade de views

___

## 1. INTRODUÇÃO

O objetivo do projeto é estimar a quantidade de visualizações que um vídeo terá com base no título, horário de publicação e as tags atribuidas aos vídeos.

___
## 2. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [88]:
import pandas as pd
import numpy as np
import json
import os

In [89]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gugat\Desktop\Insper DS 2018.1\2.1 semestre\Dados\cdados_dp\P3


### BASE DE DADOS

In [90]:
you_data = pd.read_csv("USvideos.csv")
ids = json.loads(open('US_category_id.json').read())

you_analysis = you_data.loc[you_data.views|you_data.title|you_data.tags|you_data.category_id,\
                           ['views', 'title', 'tags', 'category_id']]

you_analysis.head()


,views,title,tags,category_id
0,748374,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,22
1,2418783,The Trump Presidency: Last Week Tonight with J...,"last week tonight trump presidency|""last week ...",24
2,3191434,"Racist Superman | Rudy Mancuso, King Bach & Le...","racist superman|""rudy""|""mancuso""|""king""|""bach""...",23
3,343168,Nickelback Lyrics: Real or Fake?,"rhett and link|""gmm""|""good mythical morning""|""...",24
4,2095731,I Dare You: GOING BALD!?,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",24


### ANÁLISE DESCRITIVA

In [92]:
tags_split = you_analysis.tags.str.split(pat = '|')

for i in range(len(you_analysis)):
    for e in you_analysis[i]:
        you_analysis["tags_split"] = tags_split[i][e].replace('"', '')

you_analysis.tags_split.head()

KeyError: 0

In [ ]:
texto = [['"last week tonight donald trump"','"texto genérico"'], ['"texto genérico"']]
novo_texto = texto[0][1].replace('"', '')
print(novo_texto)

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS